# OpenStreetMap example

In this example, we download a road network from OSM using the OSMNx package, and then process the result, resulting in a RouteE Compass network dataset.

### requirements

To download an open street maps dataset, you'll need a couple of extra dependencies: [osmnx](https://osmnx.readthedocs.io/en/stable/) and [tomllib](https://docs.python.org/3/library/tomllib.html) (Python>3.11) or [toml](https://pypi.org/project/toml/) (Python<3.11). 

You can install these doing: 

```bash
pip install nrel.routee.compass[osm]
```

In [1]:
import osmnx as ox
from nrel.routee.compass.io import generate_compass_dataset
from nrel.routee.compass import CompassApp
from pathlib import Path
import os

Here we set the absolute path to output directory where you want the dataset collected:

In [2]:
output_directory = Path("golden_co")              

In this example will load in the road network that covers Golden, Colorado, as a small example but this workflow will work with any osmnx graph.

Osmnx provides many graph download operations, any are acceptable.

In [3]:
g = ox.graph_from_place("Golden, Colorado, USA")  

Now, we call the `generate_compass_dataset` function which will convert the osmnx graph into files that are compatible with routee-compass.

The network speeds arguments can also be provided here, see documentation on `generate_compass_dataset` or `osmnx.add_edge_speeds` for details.

In [4]:
generate_compass_dataset(g, output_directory)      

In order for the path arguments to work in the example TOML file, we change this Python environment's working directory to the dataset directory

In [5]:
os.chdir(output_directory)                         

If you take a look in the directory where the compass files have been written, you'll notice some `.toml` files like: `osm_default_energy.toml`. 
These are configurations for the compass application. Here's what it might look like:

```toml
parallelism = 2

[graph]
edge_list_csv = "edges-compass.csv.gz"
vertex_list_csv = "vertices-compass.csv.gz"
verbose = true

[traversal]
type = "speed_grade_energy_model"
model_type = "smartcore"
speed_table_path = "edges-posted-speed-enumerated.txt.gz"
energy_model_path = "2016_TOYOTA_Camry_4cyl_2WD.bin"
ideal_energy_rate = 0.02857142857
speed_table_speed_unit = "kilometers_per_hour"
energy_model_speed_unit = "miles_per_hour"
energy_model_grade_unit = "decimal"
energy_model_energy_rate_unit = "gallons_gasoline_per_mile"
output_time_unit = "minutes"
output_distance_unit = "miles"

[plugin]
[[plugin.input_plugins]]
type = "vertex_rtree"
vertices_file = "vertices-compass.csv.gz"

[[plugin.output_plugins]]
type = "summary"

[[plugin.output_plugins]]
type = "geometry"
route_geometry = true
tree_geometry = true
edge_file = "edges-geometries-enumerated.txt.gz"

[[plugin.output_plugins]]
type = "uuid"
uuid_file = "vertices-uuid-enumerated.txt.gz"
```

This defines the parameters for the routing engine.
See the documentation section of the config for more information.

Now we can load the application from one of our config files. 
We'll pick `osm_default_energy.toml` which will use a Toyota Camry energy model for finding the least energy route.

In [6]:
app = CompassApp.from_config_file("osm_default_energy.toml")  

uuid file: 100%|██████████| 8435/8435 [00:00<00:00, 6489784.00it/s]it/s]

Now we have an application that we can run queries against. 

In [10]:
query = {
    "origin_name": "Louis' Cafe",
    "destination_name": "Cannonball Creek Brewery",
    "origin_x": -105.2066927,
    "origin_y": 39.7348318,
    "destination_x": -105.2308575,
    "destination_y": 39.7581677,
}

In [11]:
result = app.run(query)

Taking a look at the results:

In [12]:
result["traversal_summary"]

{'distance': 3.5219894345658647,
 'distance_unit': 'miles',
 'energy': 0.1254572155432738,
 'energy_unit': 'gallons_gasoline',
 'time': 6.574806401633702,
 'time_unit': 'minutes'}